In [1]:
import numpy as np
import pandas as pd
import pickle
from IPython.display import display, clear_output
import re
from tqdm import tqdm
import string

In [2]:
dose_form_stoplist = pd.read_csv("dose-form-stoplist.tsv", sep="\t")

In [3]:
dose_form_stoplist

,dose_form
0,ACCUHALER (DISKUS)
1,AEROSOL
2,AEROSOL FOR INHALATION
3,AEROSOL INHALANT
4,AEROSOLS
...,...
602,VIAL
603,VITREOUS INJECTION
604,WAFER
605,WAFERS


In [4]:
dose_unit_stoplist = pd.read_csv("dose-unit-stoplist.tsv", sep="\t")

In [5]:
dose_unit_stoplist

,dose_unit
0,0ZS
1,1
2,1000
3,2
4,50/500
...,...
321,US
322,USP
323,UT
324,VIAL


In [27]:
# positive 
name_latin = pd.read_csv("positive-unmapped-pairs-gsrs-name-to-latin-binomial.tsv", sep="\t").dropna()
name_common = pd.read_csv("positive-unmapped-pairs-gsrs-name-to-common-name.tsv", sep="\t").dropna()
all_name = pd.read_csv("positive-unmapped-pairs-common-name-or-latin-binomial-copies.tsv", sep="\t").dropna()
mapped = pd.read_csv("NP_FAERS_mapped_20220215.csv").dropna()

# negative
neg = pd.read_csv("negative-unmapped-pairs-all.tsv", sep="\t")
neg_mapped = pd.read_csv("NP_FAERS_negative_pairs_20220222.csv")

In [7]:
mapped

,FAERS_drug_match,lookup_value
0,"ADRENAL HEALTH (BRAND) = (SIBERIAN RHODIOLA, R...",Ashwaganda
1,ASHWAGANDHA,Ashwaganda
2,"ASHWAGANDHA,",Ashwaganda
3,ASHWAGANDHA /01660201/,Ashwaganda
4,ASHWAGANDHA /01660201/,Ashwaganda
...,...,...
5353,UHE RED YEAST RICE CRANBERRY,Oryza sativa
5354,YEAST CAP RICE,Oryza sativa
5355,YEAST RICE,Oryza sativa
5356,STEVIA CLEAR,Stevia rebaudiana


In [8]:
name_latin # positive

,name,latin_binomial
0,BERBERIS SERRATIFOLIA WHOLE,BERBERIS ARISTATA
1,BERBERIS SIKKIMENSIS WHOLE,BERBERIS ARISTATA
2,BERBERIS UNDULATA WHOLE,BERBERIS ARISTATA
3,BERBERIS CERATOPHYLLA WHOLE,BERBERIS ARISTATA
4,BERBERIS ARISTATA [WHO-DD],BERBERIS ARISTATA
...,...,...
10833,CERATOCHAETE AQUATICA WHOLE,ZIZANIA AQUATICA
10834,CANADIAN WILD RICE WHOLE,ZIZANIA AQUATICA
10835,LIMNOCHLOA CADUCIFLORA WHOLE,ZIZANIA AQUATICA
10836,ZIZANIA CLAVULOSA WHOLE,ZIZANIA AQUATICA


In [9]:
name_latin.dropna()

,name,latin_binomial
0,BERBERIS SERRATIFOLIA WHOLE,BERBERIS ARISTATA
1,BERBERIS SIKKIMENSIS WHOLE,BERBERIS ARISTATA
2,BERBERIS UNDULATA WHOLE,BERBERIS ARISTATA
3,BERBERIS CERATOPHYLLA WHOLE,BERBERIS ARISTATA
4,BERBERIS ARISTATA [WHO-DD],BERBERIS ARISTATA
...,...,...
10833,CERATOCHAETE AQUATICA WHOLE,ZIZANIA AQUATICA
10834,CANADIAN WILD RICE WHOLE,ZIZANIA AQUATICA
10835,LIMNOCHLOA CADUCIFLORA WHOLE,ZIZANIA AQUATICA
10836,ZIZANIA CLAVULOSA WHOLE,ZIZANIA AQUATICA


In [43]:
name_common

,name,common_name
0,BERBERIS SERRATIFOLIA WHOLE,BARBERRY
1,BERBERIS SIKKIMENSIS WHOLE,BARBERRY
2,BERBERIS UNDULATA WHOLE,BARBERRY
3,BERBERIS CERATOPHYLLA WHOLE,BARBERRY
4,BERBERIS ARISTATA [WHO-DD],BARBERRY
...,...,...
10833,CERATOCHAETE AQUATICA WHOLE,NaN
10834,CANADIAN WILD RICE WHOLE,NaN
10835,LIMNOCHLOA CADUCIFLORA WHOLE,NaN
10836,ZIZANIA CLAVULOSA WHOLE,NaN


In [10]:
name_common.dropna()

,name,common_name
0,BERBERIS SERRATIFOLIA WHOLE,BARBERRY
1,BERBERIS SIKKIMENSIS WHOLE,BARBERRY
2,BERBERIS UNDULATA WHOLE,BARBERRY
3,BERBERIS CERATOPHYLLA WHOLE,BARBERRY
4,BERBERIS ARISTATA [WHO-DD],BARBERRY
...,...,...
194,SENNA OFFICINALIS WHOLE,SENNA
195,HINDISANA WHOLE,SENNA
196,SENNA WHOLE,SENNA
197,CASSIA SENNA WHOLE,SENNA


In [12]:
name_common_dropna = name_common.dropna()

In [19]:
name_common_dropna.columns = ['name', 'lookup_value']
name_latin.columns = ['name', 'lookup_value']
all_name.columns = ['name', 'lookup_value']
mapped.columns = ['name', 'lookup_value']

In [22]:
pos_df = pd.concat([name_common_dropna, name_latin, all_name, mapped ], ignore_index=True)

In [29]:
neg.columns = ['name', 'lookup_value']
neg_mapped.columns = ['name', 'lookup_value']

In [30]:
neg_df = pd.concat([neg, neg_mapped], ignore_index=True)

In [36]:
#pos_df.dropna() no na
# 17123 rows
pos_df.drop_duplicates()
# 17023 rows
pos_df = pos_df.drop_duplicates()


In [40]:
# neg_df.dropna() no na
# 21206 rows
neg_df.drop_duplicates()
# 21194 rows
neg_df = neg_df.drop_duplicates()

In [41]:
pos_df

,name,lookup_value
0,BERBERIS SERRATIFOLIA WHOLE,BARBERRY
1,BERBERIS SIKKIMENSIS WHOLE,BARBERRY
2,BERBERIS UNDULATA WHOLE,BARBERRY
3,BERBERIS CERATOPHYLLA WHOLE,BARBERRY
4,BERBERIS ARISTATA [WHO-DD],BARBERRY
...,...,...
17118,UHE RED YEAST RICE CRANBERRY,Oryza sativa
17119,YEAST CAP RICE,Oryza sativa
17120,YEAST RICE,Oryza sativa
17121,STEVIA CLEAR,Stevia rebaudiana


In [68]:
def concat_stoplists(stoplists):
    stop_df = pd.DataFrame(columns=["stop_word"])
    for stop in stoplists:
        stop.columns = ["stop_word"]
        stop_df = pd.concat([stop_df, stop])
    stop_df = stop_df.drop_duplicates()
    print(f"there are {stop_df.shape[0]} words")
    return stop_df

In [70]:
stop_df = concat_stoplists([dose_form_stoplist, dose_unit_stoplist])

there are 918 words


In [72]:
stop_df

,stop_word
0,ACCUHALER (DISKUS)
1,AEROSOL
2,AEROSOL FOR INHALATION
3,AEROSOL INHALANT
4,AEROSOLS
...,...
320,UNITS/ML
321,US
322,USP
323,UT


In [127]:
def remove_stop(name, stop_words):
    name = name.upper()
    for pat in stop_words['stop_word']:
        pattern = f"^{pat}(?=\s)|\s{pat}(?=\s)|\s{pat}$"
        try:
            name = re.sub(pattern, '', name)   
        except:
            pass
    name = name.strip()
    return name 

In [130]:
pos_df_sw = pos_df
for i in tqdm(range(pos_df.shape[0])):
    pos_df_sw.iloc[i, 0] = remove_stop(pos_df.iloc[i,0], stop_df)

  0%|                                                 | 0/17023 [00:00<?, ?it/s]/tmp/ipykernel_3506/1320755560.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pos_df_sw.iloc[i, 0] = remove_stop(pos_df.iloc[i,0], stop_df)
100%|███████████████████████████████████| 17023/17023 [1:03:28<00:00,  4.47it/s]


In [131]:
neg_df_sw = neg_df
for i in tqdm(range(neg_df.shape[0])):
    neg_df_sw.iloc[i, 0] = remove_stop(neg_df.iloc[i,0], stop_df)

  0%|                                                 | 0/21194 [00:00<?, ?it/s]/tmp/ipykernel_3506/2463783595.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neg_df_sw.iloc[i, 0] = remove_stop(neg_df.iloc[i,0], stop_df)
100%|███████████████████████████████████| 21194/21194 [1:19:13<00:00,  4.46it/s]


In [132]:
pos_df_sw.to_csv('pos_df_sw.csv')
neg_df_sw.to_csv('neg_df_sw.csv')

In [136]:
np.sum(pos_df_sw == pos_df)

name            17023
lookup_value    17023
dtype: int64

In [177]:
import tensorflow as tf
import string
import random
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [141]:
for name in pos_df_sw.name:
    print(name)

BERBERIS SERRATIFOLIA WHOLE
BERBERIS SIKKIMENSIS WHOLE
BERBERIS UNDULATA WHOLE
BERBERIS CERATOPHYLLA WHOLE
BERBERIS ARISTATA [WHO-DD]
BERBERIS ARISTATA DC.
BERBERIS BUSSMUL WHOLE
BERBERIS COERULESCENS WHOLE
DARVI WHOLE
BERBERIS CHITRIA WHOLE
INDIAN BARBERRY [HOC]
NEPALESE BARBERRY WHOLE
INDIAN BARBERRY WHOLE
BERBERIS GRACILLIMA WHOLE
DARUHARIDRA WHOLE
BERBERIS ARISTATA WHOLE
BERBERIS COCCINEA WHOLE
BERBERIS MACROPHYLLA WHOLE
MULDERA MULTINERVIS WHOLE
PEPPER [HOC]
BLACK PEPPER WHOLE
PIPER NIGRUM L.
PIPER NIGRUM WHOLE
CUBAN OREGANO WHOLE
INDIAN BORAGE WHOLE
COLEUS AMBOINICUS WHOLE
OCIMUM VAALAE WHOLE
COLEUS SUBORBICULARIS WHOLE
MEXICAN MINT WHOLE
COLEUS AROMATICUS WHOLE
MEXICAN THYME WHOLE
COLEUS CRASSIFOLIUS WHOLE
COLEUS VAALAE WHOLE
INDIAN BORAGE [HOC]
INDIAN THYME WHOLE
PLECTRANTHUS AMBOINICUS WHOLE
VICK'S PLANT WHOLE
COLEUS SUBFRUTECTOSUS WHOLE
COLEUS CARNOSUS WHOLE
MAJANA CARNOSA WHOLE
COLEUS SUGANDA WHOLE
MAJANA SUGANDA WHOLE
PLECTRANTHUS AMBOINICUS (LOUR.) SPRENG.
MAJANA AMBOINICA

HYPTIS POHLIANA WHOLE
NEPETA AMERICANA WHOLE
MESOSPHAERUM MENTHODORUM WHOLE
MESOSPHAERUM SPICIGERUM WHOLE
CANTINOA AMERICANA WHOLE
HYPTIS MENTHODORA WHOLE
HYPTIS GONOCEPHALA WHOLE
MESOSPHAERUM SUBVERTICILLATUM WHOLE
MESOSPHAERUM GONOCEPHALUM WHOLE
HYPTIS HISPIDA WHOLE
HYPTIS SUBVERTICILLATA WHOLE
HYPTIS LOPHANTHA WHOLE
PYCNANTHEMUM ELONGATUM WHOLE
CANTINOA AMERICANA (AUBL.) HARLEY & J.F.B.PASTORE
MESOSPHAERUM LOPHANTHUS WHOLE
LEUCAS MICROSCYPHA WHOLE
HYPTIS MADAGASCARIENSIS WHOLE
CAPPARIS ACULEATA WHOLE
CAPPARIS MICROPHYLLA WHOLE
CAPPARIS SATIVA WHOLE
CAPER BUSH [HOC]
CAPPARIS OVALIS WHOLE
CAPPARIS MURRAYI WHOLE
CAPPARIS SPINOSA L.
CAPPARIS PEDUNCULARIS WHOLE
CAPPARIS SPINOSA WHOLE
CAPPARIS OVATA WHOLE
CAPER WHOLE
CAPPARIS SPINOSA [WHO-DD]
CAPSELLA POLYMORPHA WHOLE
CAPSELLA UMENSIS WHOLE
CAPSELLA SUBALPINA WHOLE
CAPSELLA MEDITERRANEA WHOLE
CAPSELLA PERGROSSA WHOLE
CAPSELLA SELENIACA WHOLE
CAPSELLA PRAECOX WHOLE
CAPSELLA PASTORALIS WHOLE
BURSA VIGUIERI WHOLE
CAPSELLA QUERCETI WHOLE
CAPS

ASTERIAS HYBRIDA WHOLE
GENTIAN [HOC]
GENTIANUSA LUTEA WHOLE
GENTIANA LUTEA RHIZOME/WHOLE
LEXIPYRETUM LUTEUM WHOLE
GENTIANA MAJOR WHOLE
GENTIANA LUTEA L.
SWERTIA LUTEA WHOLE
ASTERIAS LUTEA WHOLE
WILD GENTIAN WHOLE
GENTIANA OLIVIERI GRISEB.
GENTIANA OLIVIERI WHOLE
GENTIANA WESCHNIAKOWII WHOLE
GENTIANODES OLIVIERI WHOLE
OLIVER'S GENTIAN [HOC]
OLIVER'S GENTIAN WHOLE
TRETORHIZA OLIVIERI WHOLE
GENTIANA REGELIANA WHOLE
DASYSTEPHANA PNEUMONANTHE WHOLE
GENTIANA EONAE WHOLE
PNEUMONANTHE VULGARIS WHOLE
GENTIANA REYESII WHOLE
GENTIANA LINIFOLIA WHOLE
CIMINALIS PSEUDOPNEUMONANTHE WHOLE
GENTIANA MANGINII WHOLE
GENTIANA MACROCARPOPHORA WHOLE
GENTIANA PNEUMONANTHE L.
LEXIPYRETUM PNEUMONANTHE WHOLE
GENTIANA ADRIANII WHOLE
GENTIANA LINEARIFOLIA WHOLE
GENTIANUSA PNEUMONANTHE WHOLE
GENTIANA NOPCSAE WHOLE
PNEUMONANTHE MINOR WHOLE
CIMINALIS PNEUMONANTHE WHOLE
GENTIANA PNEUMONANTHE WHOLE
CIMINALIS VULGARIS WHOLE
GENTIANA PNEUMONANTHOIDES WHOLE
STORKSBILL WHOLE
WILD CRANE'S-BILL WHOLE
WILD GERANIUM WHOLE
SPOT

MENTHA HIRTIFLORA WHOLE
MENTHA SUBTOMENTELLA WHOLE
PULEGIUM MICRANTHUM WHOLE
PULEGIUM TOMENTELLUM WHOLE
MENTHA TOMENTELLA WHOLE
PENNYROYAL WHOLE
PULEGIUM PULEGIUM WHOLE
PULEGIUM HETEROPHYLLUM WHOLE
MENTHA DAGHESTANICA WHOLE
MICROMERIA MARITIMA WHOLE
MENTHA ERINOIDES WHOLE
MICROMERIA FENZLII WHOLE
MENTHA PULEGIUM [HPUS]
PENNYROYAL [VANDF]
PULEGIUM VULGARE WHOLE
MENTHA AUCHERI WHOLE
PULEGIUM PUBESCENS WHOLE
MENTHA PULEGIUM
MELISSA PULEGIUM WHOLE
MENTHA PULEGIUM L.
MENTHA PULEGIOIDES WHOLE
MENTHA GIBRALTARICA WHOLE
MENTHA TOMENTOSA WHOLE
PULEGIUM DAGHESTANICUM WHOLE
MENTHA NUMIDICA WHOLE
THYMUS BIDENTATUS WHOLE
MENTHA EXIGUA WHOLE
MENTHA MONTANA WHOLE
EUROPEAN PENNYROYAL [HOC]
CHUN E BO HE WHOLE
MENTHA VIRIDIS [HPUS]
MENTHA SPICATA L.
MENTHA SPICATA SUBSP. SPICATA WHOLE
MENTHA SPICATA
SPEARMINT WHOLE
SPEARMINT HERB WHOLE
MENTHA VIRIDIS WHOLE
MENTHA SPICATA [WHO-DD]
MENTHA SPICATA WHOLE
SPEARMINT [HOC]
BUSH MINT WHOLE
ROANOKE-BELLS WHOLE
PULMONARIA VIRGINICA WHOLE
VIRGINIA COWSLIP [HOC]
LI

BORRERIA GRAMINIFOLIA WHOLE
TARDAVEL VERTICILLATA WHOLE
BORRERIA OAXACANA WHOLE
BIGELOVIA VERTICILLATA WHOLE
SPERMACOCE STELLATA WHOLE
SPERMACOCE THYMOCEPHALA WHOLE
SPHAGNUM CRASSISETUM WHOLE
SPHAGNUM SQUARROSUM CROME
SPHAGNUM SQUARROSUM WHOLE
SPHAGNUM PATULUM WHOLE
SPHYRNA LEWINI (GRIFFITH AND SMITH, 1834)
ZYGAENA INDICA WHOLE
SPHYRNA LEWINI WHOLE
ZYGAENA ERYTHRAEA WHOLE
ZYGAENA MALLEUS WHOLE
SCALLOPED HAMMERHEAD WHOLE
SPHYRNA DIPLANA WHOLE
SOUTHERN HAMMERHEAD SHARK WHOLE
SCALLOPED HAMMERHEAD SHARK WHOLE
CESTRACION OCEANICA WHOLE
CESTRACION LEEUWENII WHOLE
BRONZE HAMMERHEAD SHARK WHOLE
ZYGAENA LEWINI WHOLE
KIDNEY-HEADED SHARK WHOLE
SPHYRNA LEWENI WHOLE
STACHYS ACUMINATA WHOLE
STACHYS CALIFORNICA WHOLE
CALIFORNIA HEDGENETTLE WHOLE
STACHYS BULLATA BENTH.
STACHYS BULLATA WHOLE
STACHYS MEXICANA WHOLE
STACHYS PUBENS WHOLE
STACHYS CILIATA WHOLE
STACHYS MEXICANA BENTH.
MEXICAN HEDGENETTLE WHOLE
STACHYS EMERSONII WHOLE
STACHYS GLABRA WHOLE
STACHYS TENUIFOLIA WHOLE
STACHYS TENUIFOLIA WILLD.
SM

FLAXSEED PRIMROSE (GAMOLENIC ACID, LINOLEIC ACID, LINOLENIC ACID)
FLAXSEEDS
FLAX SEEDS
FLAX SEED SOIL
FLAXSEEDS
FLAXSEEDS +
FLAX SEEDS POWD
FLAX SEEDS
FLAXSEED SUPPLEMENT (LINUM USITATISSIMUM SEED OIL) (LINUM USITATISSIMU
FLAXSEED
FLAXSEED WITH OMEGA 3 BLEND
FLAX SEEED
FLAX SEEL
FLAX SEEN
FLAX SEEN
FLAX SEE
FLAXSEE
FLAXSEE (LENSEED OIL) (UNKNOWN)
FLAX SEE (LINUM USITATISSIMUM SEED OIL)
FLAXSEE (LINUM USITATISSIMUM SEED OIL)
FLAX SEE (LINUM USITATISSIMUM SEED OIL) (CAPSULES)
FLAXSEE
FLAX SEE AND
FLAX SSED
FLAXSSED
FLAX
FLAX XEED
GLUCOSAMINE CHONDROITIN FLAX SEED (GLUCOSAMINE, CHONDROITIN SULFATE)
GNP FLAX SEED (LINUM USITATISSIMUM SEED OIL)
GROUND FLAX
GROUND FLAX (HERBAL EXTRACT NOS)
GROUND FLAXSEED
GROUND FLAX SEED
GROUND-FLAX SEED DIET CHANGE WITH WHOLE WHEAT FOR HIGH FIBER
HERBAL SUPPLEMENT OF FLAX
HIGH LIGNAN FLAXSEED
HIGH OMEGA FLAXSEED
LINSEED (FLAXSEED OIL, GELATIN)
LINUM USITATISSIMUM       (FLAX SEED)
MAGNESIUM FLAXSEED
MILLED FLAX SEED
NATURAL ALTERNA. FLAXSEED PLUS LECITHIN


GARLIC-LECITHIN
GARLIC-PARSLEY
GARLIC/CAYENNE
GARLIC/HORSERADISH (GARLIC)
GARLIC/MULTIVITAMIN/UBIQUINONE/VITAMIN E
GARLICAP
GARLICAPS
GARLICIN
GARLICIN CARDIO
GARLICIN HIGH ALLICIN POTENTIAL
GARLIN
GARLIQUE/GARLIC
GINGER GARLIC
HAWTHORN GARLIC
HERBAL SUPPLEMENTS INCLUDING GARLIC AND ESTER
HERRON HL026 HORSERADISH AND GARLIC PLUS
HORSERADISH + GARLIC + VITAMIN C
HORSERADISH AND GARLIC
HORSERADISH, GARLIC + C
KERKERMIE TUMERIC WITH GARLIC SUPPLEMENT
KYOLIC (AGED GARLIC EXTRACT)
KYOLIC AGED GARLIC
KYOLIC AGED GARLIC EXTRACT
KYOLIC AGED GARLIC EXTRACT AND CURCUMIN
KYOLIC AGED GARLIC
NATURAL GARLIC
NATURE'S OWN GARLIC AND C HORSERADISH WITH FENUGREEK AND MARSHMALLOW
ODOR FREE GARLIC
ODOR FREE GARLIC?X
ODORLESS GARLIC                    /01570501/
OMEGA 3/GARLIC
ONION GARLIC
PARSLEY/GARLIC
POLICOSANOL GARLIC
POLICOSANOL GARLIC NIACIN
POLICOSANOL-GARLIC
RED YEAST RICE AND GARLE MAX GARLIC
SPRING VALLEY GARLIC
SUPER B, VISTARIL, GARLIC, WELLBUTRIN, FISH OIL, LEXAPRO, ESCITALOPRAM
TUMERIC WITH 

DOUBLE HEART GARLIC WITH MISTEL PLUS WHITE
FISH W/GARLIC
FOOD - PLANT SOURCE, GARLIC, ALLIUM SATIVU
GARLIC                             /01570501/
GARLIC (+) HAWTHORN (GARLIC, HAWTHORN)
GARLIC (+) PARSLEY
GARLIC (ALLIUM SATVUM)
GARLIC (ALLIUM STATIVUM)
GARLIC (ALLUM SATIVUM)
GARLIC + CAPSAICIN
GARLIC + CAPSAICIN 500
GARLIC + HORSERADISH
GARLIC + HORSERADISH (GARLIC, HORSE-RADISH)
GARLIC + PARSLEY
GARLIC + PARSLEY                   /05590701/
GARLIC + PARSLEY
GARLIC /01570501/
GARLIC /01570501/ (CON.)
GARLIC /01570504/
GARLIC AND PAIN FORMULA
GARLIC AND PARSLEY
GARLIC AND PARSLEY GELCAP
GARLIC AND PARSLEY
GARLIC AND PARSLEY
GARLIC CARE FISH
GARLIC COMPLEX
GARLIC COMPLEX (W/B6 B12 FOLIC ACID)
GARLIC LEMON W/BICARBONATE SODA
GARLIC NATURAL
GARLIC ODOR FREE
GARLIC (ALLIUM STAIVUM)
GARLIC OIL/BIPHENYL-DIMETHYL-DICARBOXYLATE
GARLIC OVER THE COUNTER
GARLIC PARSLEY
GARLIC PEARL
GARLIC PEPPER
GARLIC 16028/0058 150
GARLIC RAW
GARLIC REISHI
GARLIC REMEDIES
GARLIC ROOT
GARLIC THYROID
GARLIC W/PARSL

In [ ]:
def clean(text):
    '''remove all non-ascii, special characters and keep alphabets and space only. Can also use isalpha()
    #convert to uppercase
    #remove extra spaces'''
    regex = re.compile('[^a-zA-Z ]')
    r = regex.sub('', text)
    result = re.sub(' +', ' ', r)
    result = result.strip()
    
    return result.upper()

def clean_dataset(data):
    x = []
    y = []
    for i in tqdm(range(data.shape[0])):
        w = clean(data.iloc[i,0])
        v = clean(data.iloc[i,1])
        x.append(w)
        y.append(v)
    return x,y

def add_noise(w, percent):
    ''' edit, del, add'''
    positions = random.choices(range(len(w)), k=int(percent*len(w)))
    for p in positions:
        r = random.random()
        if r <= 1/3: # edit
            w = w[:p] + random.choice(string.ascii_uppercase) + w[p+1:]
        elif r<= 2/3: # delete
            w = w[:p] + w[p+1:]
        elif r<=1: # add
            w = w[:p] + random.choice(string.ascii_uppercase) + w[p:]       
    return w

def add_noise_dataset(X, Y, multiply=10, noise_percentage= 0.2):
    x = []
    y = []
    for i in tqdm(range(len(X))):
        v = Y[i]
        for _ in range(multiply):
            w = add_noise(X[i], noise_percentage)
            x.append(w)
            y.append(v)
    return x,y

def encode_dataset(x,y, encode_dict):
    encode_dict = {l:i+1 for i,l in enumerate(string.ascii_uppercase + " ")}
    Xtrain = [[encode_dict[m] for m in n] for n in x]
    Ytrain = [[encode_dict[m] for m in n] for n in y]
    return Xtrain, Ytrain

def padding_dataset(X,Y,maxlen):
    padded_y = tf.keras.preprocessing.sequence.pad_sequences(
        Y, padding="post", maxlen=maxlen)
    padded_x = tf.keras.preprocessing.sequence.pad_sequences(
        X, padding="post", maxlen=maxlen)
    return padded_x, padded_y

In [171]:
x_pos, y_pos = clean_dataset(pos_df_sw)
print("x,y pos", len(x_pos), len(y_pos))
x_pos_noise, y_pos_noise = add_noise_dataset(x_pos, y_pos)
print("x,y pos with noise", len(x_pos_noise), len(y_pos_noise))
x_neg, y_neg = clean_dataset(neg_df_sw)
print("x,y neg", len(x_neg), len(y_neg))
x_neg_noise, y_neg_noise = add_noise_dataset(x_neg, y_neg)
print("x,y neg with noise", len(x_neg_noise), len(y_neg_noise))

100%|██████████████████████████████████| 17023/17023 [00:01<00:00, 11672.04it/s]


x,y pos 17023 17023


100%|██████████████████████████████████| 17023/17023 [00:01<00:00, 10610.48it/s]


x,y pos with noise 170230 170230


100%|██████████████████████████████████| 21194/21194 [00:01<00:00, 11697.34it/s]


x,y neg 21194 21194


100%|███████████████████████████████████| 21194/21194 [00:02<00:00, 9419.93it/s]

x,y neg with noise 211940 211940


In [178]:
encode_dict = {l:i+1 for i,l in enumerate(string.ascii_uppercase + " ")}
XPos, YPos = encode_dataset(x_pos_noise, y_pos_noise, encode_dict)
x_pos_padded, y_pos_padded = padding_dataset(XPos, YPos, maxlen=400)
XNeg, YNeg = encode_dataset(x_neg_noise, y_neg_noise, encode_dict)
x_neg_padded, y_neg_padded = padding_dataset(XNeg, YNeg, maxlen=400)

In [199]:
ind = [i for i in range(len(XPos))]
label = np.zeros(len(XPos)+len(XNeg))
label[ind] = 1
label

array([1., 1., 1., ..., 0., 0., 0.])

In [201]:
X = np.concatenate([x_pos_padded, x_neg_padded])
Y = np.concatenate([y_pos_padded, y_neg_padded])
X_str = x_pos_noise + x_neg_noise
Y_str = y_pos_noise + y_neg_noise

In [202]:
Xs, Ys, X_strs, Y_strs, labels = shuffle(X, Y, X_str, Y_str, label, random_state=42)

In [229]:
Xtrain, Xval, Ytrain, Yval, Xtrain_str, Xval_str, Ytrain_str, Yval_str, labeltrain, labelval = train_test_split(Xs,Ys,X_strs,Y_strs,labels,test_size=0.15, random_state=42)

# Hyperparam tuning 

In [207]:
import skopt
from skopt import BayesSearchCV
from skopt import gp_minimize, forest_minimize
from skopt.space import Real, Categorical, Integer
from skopt.plots import plot_convergence
from skopt.plots import plot_objective, plot_evaluations
from skopt.plots import plot_histogram, plot_objective_2D
from skopt.utils import use_named_args

In [234]:
def cosine_distance(vects):
    x, y = vects
    return 1-tf.reduce_sum(tf.multiply(x,y),axis=1, keepdims=True)/(tf.norm(x,axis=1,keepdims=True)*tf.norm(y,axis=1,keepdims=True))

def loss(margin=1):
    def contrastive_loss(y_true, y_pred):
        square_pred = tf.math.square(y_pred)
        margin_square = tf.math.square(tf.math.maximum(margin - (y_pred), 0))
        return tf.math.reduce_mean(
            (1 - y_true) * square_pred + (y_true) * margin_square
        )

    return contrastive_loss

In [203]:
tf.config.list_physical_devices()

2022-04-19 01:09:58.903870: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-04-19 01:09:59.031733: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-04-19 01:09:59.364146: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:af:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2022-04-19 01:09:59.364214: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-04-19 01:10:01.763813: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-04-19 01:10:01.763959: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [232]:
def build_model(model_type, embedding_dim, num_rnn_node, num_dense_node, num_layer, activation_fn, learning_rate, optimizer, margin):
    maxlen =400
    input_x = tf.keras.layers.Input(maxlen)
    input_1 = tf.keras.layers.Input(maxlen)
    input_2 = tf.keras.layers.Input(maxlen)
    embedding = tf.keras.layers.Embedding(input_dim=28, output_dim=embedding_dim, mask_zero=True)
    x = embedding(input_x)
    
    if model_type == "lstm":
        x = tf.keras.layers.LSTM(num_rnn_node)(x)
    elif model_type=="gru":
        x = tf.keras.layers.GRU(num_rnn_node)(x)
 
    num = num_dense_node
    for _ in range(num_layer):
        x = tf.keras.layers.Dense(num, activation=activation_fn)(x)
        num /= 2
        
    embedding_network = tf.keras.Model(input_x, x)

    tower_1 = embedding_network(input_1)
    tower_2 = embedding_network(input_2)

    merge_layer = tf.keras.layers.Lambda(cosine_distance)([tower_1, tower_2])
    output_layer = tf.keras.layers.Dense(1, activation="sigmoid")(merge_layer)
    contr = tf.keras.Model(inputs=[input_1, input_2], outputs=output_layer)
    
    if optimizer == "Adam":
        opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer =="RMSprop":                
        opt = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    contr.compile(loss=loss(margin= margin), optimizer=opt, metrics=["accuracy"])
    return contr

In [236]:
#dim_model = Categorical(categories=["lstm", "gru"], name="model_type")
dim_model = Categorical(categories=["lstm"], name="model_type")
dim_embedding = Integer(low=64, high=512, prior="log-uniform", base=2, name="embedding_dim")
dim_rnn_node = Integer(low=128, high=1024, prior="log-uniform", base=2, name="num_rnn_node")
dim_num_dense_nodes = Integer(low=64, high=512, prior="log-uniform", base=2, name="num_dense_node")
dim_num_layer = Integer(low=1, high=5, prior="uniform", name="num_layer")
dim_activation = Categorical(categories=['tanh', 'relu'], name="activation_fn")
dim_lr = Real(low=1e-5, high=1e-1, prior="log-uniform", base=10, name="learning_rate")
dim_opt = Categorical(categories=['Adam', 'RMSprop'], name="optimizer")
dim_margin = Real(low=1e-1, high=1, prior="uniform", name="margin")
dim_batch_size = Integer(low=4, high=64, prior="log-uniform", base=2, name="batch_size")
dims = [dim_model, dim_embedding, dim_rnn_node, dim_num_dense_nodes, dim_num_layer, dim_activation, dim_lr, dim_opt, dim_margin, dim_batch_size]
default_params = ["gru", 64, 128, 64, 2, "tanh", 1e-3, "RMSprop", 1, 32]

In [221]:
def log_dir_name(a):
    o = ""
    for n in a:
        o+= str(n)+"_"
    return o+".log"

In [230]:
@use_named_args(dimensions=dims)
def fitness(model_type, embedding_dim, num_rnn_node, num_dense_node, num_layer, activation_fn, learning_rate, optimizer, margin, batch_size):
    
    print("model:", model_type)
    print("embedding_dim:", embedding_dim)
    print("num_rnn_node:", num_rnn_node)
    print("num_dense_node:", num_dense_node)
    print("num_layer:", num_layer)
    print("activation_fn:", activation_fn)
    print("learning rate: {:.1e}".format(learning_rate))
    print("optimizer:", optimizer)
    print("margin:", margin)
    print("batch_size:", batch_size)

    model = build_model(model_type, embedding_dim, num_rnn_node, num_dense_node, num_layer, activation_fn, learning_rate, optimizer, margin)
    params = [model_type, embedding_dim, num_rnn_node, num_dense_node, num_layer, activation_fn, learning_rate, optimizer, margin, batch_size]
   
    history = model.fit(x=[Xtrain, Ytrain],
                        y=labeltrain,
                        epochs=40,
                        batch_size=batch_size,
                        validation_data=([Xval, Yval], labelval),
                        )
    
    loss = history.history['val_loss'][-1]
    print("Val loss: {0:.6%}".format(loss))
    model_name = "_".join([str(a) for a in params]) + f"{str(loss).replace('.', '_')}.h5"
    model.save(f"exp3_model/{model_name}")
    del model    
    
    return loss

In [225]:
save_model = tf.keras.callbacks.ModelCheckpoint(
      filepath= "exp5_model/alstm" + "-{epoch:02d}-{val_loss:.4f}.hdf5",
      save_weights_only=True,
      monitor='val_loss',
      verbose=1,
      save_best_only=False)

In [226]:
@use_named_args(dimensions=dims)
def fitness2(model_type, embedding_dim, num_rnn_node, num_dense_node, num_layer, activation_fn, learning_rate, optimizer, margin, batch_size):
    
    print("model:", model_type)
    print("embedding_dim:", embedding_dim)
    print("num_rnn_node:", num_rnn_node)
    print("num_dense_node:", num_dense_node)
    print("num_layer:", num_layer)
    print("activation_fn:", activation_fn)
    print("learning rate: {:.1e}".format(learning_rate))
    print("optimizer:", optimizer)
    print("margin:", margin)
    print("batch_size:", batch_size)

    model = build_model(model_type, embedding_dim, num_rnn_node, num_dense_node, num_layer, activation_fn, learning_rate, optimizer, margin)
    params = [model_type, embedding_dim, num_rnn_node, num_dense_node, num_layer, activation_fn, learning_rate, optimizer, margin, batch_size]
   
    history = model.fit(x=[np.array(x1TrainRnnS), np.array(x2TrainRnnS)],
                        y=np.array(yTrainRnnS, dtype=np.float32),
#                         epochs=35,
                        epochs=30,
                        batch_size=batch_size,
                        validation_data=([np.array(x1ValRnnS), np.array(x2ValRnnS)], np.array(yValRnnS, dtype=np.float32)),
                        callbacks=[save_model])
    
    loss = history.history['val_loss'][-1]
    print("Val loss: {0:.6%}".format(loss))
    model_name = "_".join([str(a) for a in params]) + f"{str(loss).replace('.', '_')}.h5"
    model.save(f"exp3_model/{model_name}")
    del model    
    
    return loss

In [227]:
fitness(x=default_params)

model: gru
embedding_dim: 64
num_rnn_node: 128
num_dense_node: 64
num_layer: 2
activation_fn: tanh
learning rate: 1.0e-03
optimizer: RMSprop
margin: 1
batch_size: 32


NameError: name 'maxlen' is not defined

In [237]:
#search_result = gp_minimize(func=fitness, dimensions=dims, acq_func='EI', n_calls=40, x0=default_params)
search_result = gp_minimize(func=fitness, dimensions=dims, acq_func='EI', n_calls=100)

model: lstm
embedding_dim: 219
num_rnn_node: 136
num_dense_node: 399
num_layer: 3
activation_fn: relu
learning rate: 7.3e-04
optimizer: RMSprop
margin: 0.6259390901650306
batch_size: 42
Epoch 1/40
7735/7735 [==============================] - 391s 49ms/step - loss: 0.1055 - accuracy: 0.5617 - val_loss: 0.0884 - val_accuracy: 0.5937
Epoch 2/40
7735/7735 [==============================] - 367s 47ms/step - loss: nan - accuracy: 0.5967 - val_loss: nan - val_accuracy: 0.5538
Epoch 3/40
7735/7735 [==============================] - 374s 48ms/step - loss: nan - accuracy: 0.5553 - val_loss: nan - val_accuracy: 0.5538
Epoch 4/40
7735/7735 [==============================] - 372s 48ms/step - loss: nan - accuracy: 0.5540 - val_loss: nan - val_accuracy: 0.5538
Epoch 5/40
7735/7735 [==============================] - 365s 47ms/step - loss: nan - accuracy: 0.5541 - val_loss: nan - val_accuracy: 0.5538
Epoch 6/40
7735/7735 [==============================] - 378s 49ms/step - loss: nan - accuracy: 0.5544 -

17098/17098 [==============================] - 673s 39ms/step - loss: 0.0571 - accuracy: 0.5562 - val_loss: 0.0572 - val_accuracy: 0.5538
Epoch 29/40
17098/17098 [==============================] - 655s 38ms/step - loss: 0.0572 - accuracy: 0.5538 - val_loss: 0.0572 - val_accuracy: 0.5538
Epoch 30/40
17098/17098 [==============================] - 657s 38ms/step - loss: 0.0571 - accuracy: 0.5567 - val_loss: 0.0572 - val_accuracy: 0.5538
Epoch 31/40
17098/17098 [==============================] - 660s 39ms/step - loss: 0.0571 - accuracy: 0.5560 - val_loss: 0.0572 - val_accuracy: 0.5538
Epoch 32/40
17098/17098 [==============================] - 598s 35ms/step - loss: 0.0572 - accuracy: 0.5548 - val_loss: 0.0572 - val_accuracy: 0.5538
Epoch 33/40
17098/17098 [==============================] - 599s 35ms/step - loss: 0.0572 - accuracy: 0.5546 - val_loss: 0.0572 - val_accuracy: 0.5538
Epoch 34/40
17098/17098 [==============================] - 604s 35ms/step - loss: 0.0572 - accuracy: 0.5541 - va

Epoch 1/40
10152/10152 [==============================] - 397s 38ms/step - loss: nan - accuracy: 0.5596 - val_loss: nan - val_accuracy: 0.5538
Epoch 2/40
10152/10152 [==============================] - 386s 38ms/step - loss: nan - accuracy: 0.5573 - val_loss: nan - val_accuracy: 0.5538
Epoch 3/40
10152/10152 [==============================] - 385s 38ms/step - loss: nan - accuracy: 0.5554 - val_loss: nan - val_accuracy: 0.5538
Epoch 4/40
10152/10152 [==============================] - 385s 38ms/step - loss: nan - accuracy: 0.5548 - val_loss: nan - val_accuracy: 0.5538
Epoch 5/40
10152/10152 [==============================] - 386s 38ms/step - loss: nan - accuracy: 0.5549 - val_loss: nan - val_accuracy: 0.5538
Epoch 6/40
10152/10152 [==============================] - 383s 38ms/step - loss: nan - accuracy: 0.5554 - val_loss: nan - val_accuracy: 0.5538
Epoch 7/40
10152/10152 [==============================] - 385s 38ms/step - loss: nan - accuracy: 0.5564 - val_loss: nan - val_accuracy: 0.5538

20303/20303 [==============================] - 607s 30ms/step - loss: 0.1018 - accuracy: 0.5663 - val_loss: 0.1016 - val_accuracy: 0.5780
Epoch 31/40
20303/20303 [==============================] - 594s 29ms/step - loss: 0.1026 - accuracy: 0.5643 - val_loss: 0.1110 - val_accuracy: 0.5853
Epoch 32/40
20303/20303 [==============================] - 587s 29ms/step - loss: 0.1026 - accuracy: 0.5623 - val_loss: 0.1022 - val_accuracy: 0.5538
Epoch 33/40
20303/20303 [==============================] - 588s 29ms/step - loss: 0.1024 - accuracy: 0.5618 - val_loss: 0.1047 - val_accuracy: 0.5899
Epoch 34/40
20303/20303 [==============================] - 585s 29ms/step - loss: 0.1026 - accuracy: 0.5624 - val_loss: 0.1011 - val_accuracy: 0.5538
Epoch 35/40
20303/20303 [==============================] - 589s 29ms/step - loss: 0.1023 - accuracy: 0.5621 - val_loss: 0.1037 - val_accuracy: 0.5538
Epoch 36/40
20303/20303 [==============================] - 587s 29ms/step - loss: 0.1025 - accuracy: 0.5619 - va

40606/40606 [==============================] - 1141s 28ms/step - loss: 0.0267 - accuracy: 0.5545 - val_loss: 0.0266 - val_accuracy: 0.5538
Epoch 17/40
40606/40606 [==============================] - 1122s 28ms/step - loss: 0.0267 - accuracy: 0.5541 - val_loss: 0.0271 - val_accuracy: 0.5538
Epoch 18/40
40606/40606 [==============================] - 1000s 25ms/step - loss: 0.0267 - accuracy: 0.5552 - val_loss: 0.0267 - val_accuracy: 0.5538
Epoch 19/40
40606/40606 [==============================] - 1144s 28ms/step - loss: 0.0267 - accuracy: 0.5534 - val_loss: 0.0266 - val_accuracy: 0.5538
Epoch 20/40
40606/40606 [==============================] - 1196s 29ms/step - loss: 0.0267 - accuracy: 0.5546 - val_loss: 0.0268 - val_accuracy: 0.5538
Epoch 21/40
40606/40606 [==============================] - 1219s 30ms/step - loss: 0.0267 - accuracy: 0.5547 - val_loss: 0.0266 - val_accuracy: 0.5538
Epoch 22/40
40606/40606 [==============================] - 1202s 30ms/step - loss: 0.0267 - accuracy: 0.55

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
dims = [dim_model, dim_embedding, dim_rnn_node, dim_num_dense_nodes, dim_num_layer, dim_activation, dim_lr, dim_opt, dim_margin, dim_batch_size]
default_params = ["gru", 64, 128, 64, 2, "tanh", 1e-3, "RMSprop", 1, 32]
best_params = ["lstm", 256, 512, 256, 1, "tanh", 2e-4, "Adam", 0.8, 4]

In [ ]:
fitness2(x=best_params)